# Coop:: Get citations from Google Scholar using a title list on .csv

## Imports

In [7]:
import numpy as np
import pandas as pd
import requests
import time 
import random 
import datetime
import ipywidgets as widgets

## Read biblio from Scopus CVS

In [8]:
df = pd.read_csv('ddos_coop_collab_.csv')
#df = pd.read_csv('honeypot_googlescholar_query_honeypot_20190722.csv',index_col=0).dropna()

In [9]:
print("TOTAL ENTRIES:",len(df))
df.head(2)

TOTAL ENTRIES: 343


,abstract,doi,eid,link,scholar,scopus,title,year
0,Distributed denial-of-service (DDoS) attacks h...,10.1109/ICON.2002.1033348,2-s2.0-33745278421,https://www.scopus.com/inward/record.uri?partn...,NaN,27.0,Engineering of a global defense infrastructure...,2002
1,"© 2019 John Wiley & Sons, Ltd.The amount of wi...",10.1002/dac.4241,2-s2.0-85075171039,https://www.scopus.com/inward/record.uri?partn...,NaN,0.0,Dual-collaborative DoS/DDoS mitigation approac...,2020


## Open Google Scholar and type query

### Use Selenium - I think is better way to solve captchas

In [14]:
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
#from selenium.common.exceptions import *

# get the path of ChromeDriverServer 
chrome_driver_path = "/usr/local/bin/chromedriver"

# create a new Chrome session and open Google Scholar page
driver = webdriver.Chrome(chrome_driver_path)
driver.get("https://scholar.google.com")

# Find textbox and submit text
#search_field = driver.find_element_by_name("q")
#search_field.send_keys("a search term\n")

### Get citations on GOOGLE SCHOLAR from SCOPUS selection

In [15]:
import requests
from bs4 import BeautifulSoup as bs
import time 
import random 
import datetime
import re

In [16]:
scholar_url = 'https://scholar.google.com/scholar?as_sdt=0%2C5&btnG=&hl=en&inst=5726176096060060532&q='
msg_cited = ['Cited', 'Geciteerd', 'Citado']
recaptcha = False

bar = widgets.IntProgress(
    min=0, 
    max=len(df), 
    description='Loading')
display(bar)

for i in df.index:
    bar.value += 1
    bar.description = str(bar.value)
    
    if (pd.notnull(df.scholar.loc[i])):
        #print ('[', i , ']', '...skipping..')
        continue
        
    p = df.loc[i]
    url = scholar_url + '"'+ p.title + '"'
    driver.get(url)
    
    soup = bs(driver.page_source, 'html.parser')
    
    # Must change here to multilingual too
    cited_by = soup.select_one('a:contains("Cited by")').text if soup.select_one('a:contains("Cited by")') is not None else 'No citation count'
    #print (cited_by)

    if (cited_by.split()[0] in msg_cited) :
        # Citation found (update dataframe)
        scholar_citation = re.sub('[^0-9]', '', cited_by)
        df.at[i,'scholar']=scholar_citation

    else:
        # Citation not found (can be lang, captcha or zero citation)
        scholar_citation = 0
        recaptcha = True
        
        #some_google_answers = ['Related', 'results', 'entire web', 'Suggestions', 'versions', 'Related articles']
        #for txt in some_google_answers:
        #    if ( soup.select_one('a:contains(txt)')) :
        #        # There is no citations for sure
        #        recaptcha = False 
        #        df.at[i,'scholar']=scholar_citation
        #        print ('Found: ', txt)
           
        #if ( soup.find (text=re.compile(re.escape(p.title))) ):
        if ( soup.find(text=re.compile( re.escape(p.title[:20]),re.I)) ):
            # I have title, zero citation = not in recaptcha
            recaptcha = False 
            df.at[i,'scholar']=scholar_citation
        
    #print (scholar_citation)  
    print ('[', i , recaptcha, ']', p.scopus, scholar_citation, p.title[0:20], sep="\t")
    
    #time.sleep(random.randint(5, 30))

IntProgress(value=0, description='Loading', max=343)

[	159	False	]	372.0	547	The effective temper
[	160	False	]	3.0	5	Bionic autonomic ner
[	161	False	]	5.0	15	IFTS: Intrusion fore
[	162	False	]	54.0	120	Mitigating bandwidth
[	163	False	]	0.0	2	A Framework for Data
[	164	False	]	12.0	16	OverWatch: A Cross-P
[	165	False	]	1.0	2	A firegroup mechanis
[	166	False	]	83.0	152	FireCol: A collabora
[	167	False	]	1.0	1	Heterogeneous tracer
[	168	False	]	0.0	0	Research of MDCOP mi
[	169	False	]	1.0	0	DG-based DDoS detect
[	170	False	]	11.0	18	Detecting and preven
[	171	False	]	1.0	2	Cooperative security
[	172	False	]	0.0	3	Defending against ev
[	173	False	]	0.0	2	Privacy-preserving a
[	174	False	]	1.0	4	Botnet detection: A 
[	175	False	]	29.0	54	An active detecting 
[	176	False	]	15.0	21	Collaborative anomal
[	177	False	]	7.0	13	AutoBotCatcher: Bloc
[	178	False	]	33.0	60	Collaborative change
[	179	False	]	0.0	1	Unfair rate limiting
[	180	False	]	10.0	50	Honeypot back-propag
[	181	False	]	5.0	14	Stellar: Network Att
[	182	False	]	0.0	3	Study of dec

In [17]:
df.to_csv('ddos_coop_collab_citations.csv',index=False)